---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kpcrotty.github.io/)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [1]:
S = 100         # underlying price
K = 100         # strike price
T = 1           # years to maturity
sigma = 0.4     # volatility of underlying
r = 0.02        # risk-free rate
q = 0.04        # dividend yield
N = 20000       # number of simulations

# SIMULATIONS

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Black-Scholes values
d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
N1 = norm.cdf(d1)
d2 = d1 - sigma * np.sqrt(T)
N2 = norm.cdf(d2)
call_bs = np.exp(-q*T)*S*N1 - np.exp(-r*T)*K*N2
put_bs  = np.exp(-r*T)*K*(1-N2) - np.exp(-q*T)*S*(1-N1)

# Simulated values
eps = norm.rvs(0, 1, N)

# future stock price
ST = S * np.exp((r - q - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * eps)

# call and put values if in the money at maturity
calls = [s for s in ST if s>K]
puts  = [s for s in ST if s<K]

# Monte carlo values of call and put
call_monte = np.exp(-r*T)*pd.Series(np.maximum(ST-K,0)).mean()
put_monte  = np.exp(-r*T)*pd.Series(np.maximum(K-ST,0)).mean()

# Summarize
print(f'Black-Scholes call value:\t${call_bs:.2f}')
print(f'Monte Carlo call value:\t\t${call_monte:.2f}\n')
print(f'Black-Scholes put value:\t${put_bs:.2f}')
print(f'Monte Carlo put value:\t\t${put_monte:.2f}')

Black-Scholes call value:	$14.43
Monte Carlo call value:		$14.50

Black-Scholes put value:	$16.37
Monte Carlo put value:		$16.41


# FIGURE (UNDERLYING AT MATURITY)

In [3]:
import plotly.graph_objects as go

trace1 = go.Histogram(x=calls, name="call in the money", marker=dict(color='blue'))
trace2 = go.Histogram(x=puts, name="put in the money", marker=dict(color='red'))
fig = go.Figure()
fig.add_trace(trace2)
fig.add_trace(trace1)
fig.update_xaxes(title = "Underlying Price at Maturity")
fig.update_yaxes(tickvals=[])
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))
fig.show()


# FIGURE (IN THE MONEY PUT)

In [4]:
fig_put = go.Figure(go.Histogram(x=K-np.array(puts), 
                                marker=dict(color='red')
                                )
                    )
fig_put.update_yaxes(tickvals=[])
fig_put.update_xaxes(title = "Put Value When in the Money")
fig_put.show()

# FIGURE (IN THE MONEY CALL)

In [5]:
fig_call = go.Figure(go.Histogram(x=np.array(calls)-K, 
                                marker=dict(color='blue')
                                )
                    )
fig_call.update_yaxes(tickvals=[])
fig_call.update_xaxes(title = "Call Value When in the Money")
fig_call.show()
